In [1]:
import load
from plot import plot3D

# from mygp import mygp

import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
from sklearn.preprocessing import StandardScaler

import skopt 
from skopt.space import Real


from icecream import ic





In [2]:
class mygp(object):

    def __init__(
        self,  
        gp=None,  
        **kwargs    
    ):
    
        if gp:
            self.gp = gp    
        else:
            self.gp = GaussianProcessRegressor()    



        self.ndims=1
        self.sample_X = np.array([])
        self.sample_y = np.array([])

        
        self.kwargs = kwargs
        self.kappa = kwargs.pop("kappa",0.0)
        self.acquisition_function = kwargs.pop("acquisition_function","LCB")    



    def prn(self, **kwargs):


        print(f"ndims = {self.ndims}. sample shape = {self.sample_X.shape}")
        ic(self.sample_X, self.sample_y)
        print(f"kappa={self.kappa}")
        print(f"acquisition_function={self.acquisition_function}")    

        return

    def sample_init(self,X,y):

        X_shape = X.shape
        X_len =  len(X_shape)

        
        if X_len == 1:
            self.ndims = 1
        elif X_len == 2:
            self.ndims = X_shape[1]
        else:         
            raise ValueError(f'sample_init() sample_X has incompatiple shape"{X_shape}" for the gaussian fit method.')
        
        self.sample_X = X.copy()
        self.sample_y = y.copy()


        self.gp.fit(self.sample_X, self.sample_y)

        sample_pred , sample_std = self.gp.predict(self.sample_X, return_std=True)

        return sample_pred

    def predict(self, grid_X):
                           
        X_len = len(grid_X.shape)

        if X_len == 1 and self.ndims == 1:
            pass      
        elif X_len ==2 and grid_X.shape[1] == self.ndims:       
            pass        
        else:
            raise ValueError(f'predict() grid_X has shape "{grid_X.shape}" dimensions. ndims = {self.ndims}')   

        grid_pred, grid_std = self.gp.predict(grid_X, return_std=True)

        return grid_pred, grid_std

    def log_marginal_likelihood(self, theta=None,eval_gradient=False, clone_kernel=True):

        return self.gp.log_marginal_likelihood( theta=theta, eval_gradient=eval_gradient,clone_kernel=clone_kernel  )
    
    def get_params(self):
        ic(self.gp.get_params(deep=True))

    def sample_add(self, X1, y1):

        if len(X1) != self.ndims: 
            raise ValueError(f'sample_add(). Error sample_X1 must have exactly "{self.ndims}"')      

        # add point to array
        X = np.concatenate((self.sample_X, [X1]),axis=0 )
        y = np.concatenate((self.sample_y, [y1]),axis=0)     

        ic(X.shape,y.shape)
        ic(X,y)
        
        # remember if sucessfull this will update self.sample_X, self.sample_y
        self.sample_init(X,y)

        return
    
    def gp_minimize(self):

        def surrogate_function(inputs):
            # Predict using the Gaussian Process surrogate model
            y_pred, stdev = self.gp.predict(np.atleast_2d(inputs), return_std=True) 
            return y_pred[0]  



        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    
        
        res = skopt.gp_minimize(
            func=surrogate_function,
            dimensions=my_domain,  # The search space
            # base_estimator= self.gp, 
            acq_func=self.acquisition_function,
            kappa= self.kappa,
            n_calls = 4,  # high to be on the safe side
            
            x0=[list(x) for x in self.sample_X], # Sample X values
            y0= self.sample_y,                   # y target 
            verbose=False                         # debug output ??
        )
        
        print(f"gp_minimize() res.x = {res.x}")
        print(f"gp_minimize() res.fun = {res.fun}")

        suggested = np.array(res.x_iters[-1])
        print("Suggested next inputs to try:", suggested)

        del res  

        return





a,b = load_function( 1,2 )
ic(a.shape,b.shape)
ic(a,b)


In [ ]:
kernel = kernels.RBF(length_scale=0.08)

g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)

gp = mygp(g, kappa=10.0, acquisition_function="LCB" )

# load.print_for_capstone()


sample_X, sample_y = load.load_function(1,0)

# sample_y*=-1.0

gp.sample_init(sample_X,sample_y)

gp.prn()

# gp.get_params()

gp.gp_minimize()


ndims = 2. sample shape = (10, 2)


ic| self.sample_X: array([[0.31940389, 0.76295937],
                          [0.57432921, 0.8798981 ],
                          [0.73102363, 0.73299988],
                          [0.84035342, 0.26473161],
                          [0.65011406, 0.68152635],
                          [0.41043714, 0.1475543 ],
                          [0.31269116, 0.07872278],
                          [0.68341817, 0.86105746],
                          [0.08250725, 0.40348751],
                          [0.88388983, 0.58225397]])
    self.sample_y: array([ 1.32267704e-079,  1.03307824e-046,  7.71087511e-016,
                           3.34177101e-124, -3.60606264e-003, -2.15924904e-054,
                          -2.08909327e-091,  2.53500115e-040,  3.60677119e-081,
                           6.22985647e-048])


kappa=10.0
acquisition_function=LCB


ValueError: Expected `n_calls` >= 10, got 4

: 

In [ ]:
ndims = 2. sample shape = (12, 2)
kappa=10.0
acquisition_function=LCB
gp_minimize() res.x = [0.10397175087275337, 0.5684423547821214]
gp_minimize() res.fun = -0.15683672515706348
Suggested next inputs to try: [0.10397175 0.56844235]

In [3]:
x1, x2 = np.meshgrid(np.linspace( 0.0, 1.0, 100), np.linspace(0.0, 1.0, 100))
X_plot = np.vstack([x1.ravel(), x2.ravel()]).T

ic(x1.shape,x2.shape,X_plot.shape)

# Predict on the meshgrid
y_pred, y_std = gp.predict(X_plot)    


ic(np.min(y_pred), np.max(y_pred))
y_pred_plot = y_pred.reshape(x1.shape)
y_std_plot  = y_std.reshape(x1.shape)

ic(y_pred.shape, y_std.shape, y_pred_plot.shape, y_std_plot.shape)

plot3D(x1,x2,y_pred_plot+y_std_plot)
plot3D(x1,x2,y_std_plot)

ic| x1.shape: (100, 100)
    x2.shape: (100, 100)
    X_plot.

shape: (10000, 2)
ic| np.min(y_pred): -0.48709583434629167
    np.max(y_pred): 0.46658415293404687
ic| y_pred.shape: (10000,)
    y_std.shape: (10000,)
    y_pred_plot.shape: (100, 100)
    y_std_plot.shape: (100, 100)
/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/IPython/core/display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [6]:
from sklearn.decomposition import PCA
pca = PCA(2).fit(sample_X)
